In [1]:
from  numba import njit
import numpy as np
import pickle
from src.envs.utils import GraphDataset

In [2]:
@njit
def flatten_graph(graph):
    """
    Flatten a graph into matrices for adjacency, weights, start indices, and end indices.

    Parameters:
    - graph (adjacency matrix): The input graph to be flattened.

    Returns:
    - numpy.ndarray: Flattened adjacency matrix.
    - numpy.ndarray: Flattened weight matrix.
    - numpy.ndarray: Start indices for nodes in the flattened matrices.
    - numpy.ndarray: End indices for nodes in the flattened matrices.
    """
    flattened_adjacency = []
    flattened_weights = []
    num_nodes = graph.shape[0]
    
    node_start_indices = np.zeros(num_nodes,dtype=np.int64)
    node_end_indices = np.zeros(num_nodes,dtype=np.int64)
    
    for i in range(num_nodes):
        node_start_indices[i] = len(flattened_adjacency)
        for j in range(num_nodes):
            if graph[i, j] != 0:
                flattened_adjacency.append(j)
                flattened_weights.append(graph[i, j])
                
        node_end_indices[i] = len(flattened_adjacency)

    return (
        np.array(flattened_adjacency),
        np.array(flattened_weights),
        node_start_indices,
        node_end_indices
    )





In [3]:
@njit
def standard_greedy(graph):
    adj_matrix, weight_matrix, start_list, end_list=graph
    
    n=len(start_list)
    delta_local_cuts=np.zeros(n)
    spins=np.ones(n)
    
    
    curr_score=0
    for i in range(n):
        for j,weight in zip(adj_matrix[start_list[i]:end_list[i]],
                  weight_matrix[start_list[i]:end_list[i]]):
                
            delta_local_cuts[i]+=weight*(2*spins[i]-1)*(2*spins[j]-1)
            curr_score+=weight*(spins[i]+spins[j]-2*spins[i]*spins[j])

    curr_score/=2    
    best_score=curr_score
    
    flag=True
    
    while flag:
        arg_gain=np.argsort(-delta_local_cuts)
        flag=False
        for v in arg_gain:
            if spins[v]:
                if delta_local_cuts[v]<0:
                    flag=False
                    break
                    
                curr_score+=delta_local_cuts[v]
                delta_local_cuts[v]=-delta_local_cuts[v]
                
                for u,weight in zip(adj_matrix[start_list[v]:end_list[v]],
                                     weight_matrix[start_list[v]:end_list[v]]):

                    delta_local_cuts[u]+=weight*(2*spins[u]-1)*(2-4*spins[v])

                spins[v] = 1-spins[v]
                flag=True
                break
                  
    return curr_score,spins

In [4]:
@njit
def mca(graph,spins):
    adj_matrix, weight_matrix, start_list, end_list=graph
    
    n=len(start_list)
    delta_local_cuts=np.zeros(n)
    
    
    
    curr_score=0
    for i in range(n):
        for j,weight in zip(adj_matrix[start_list[i]:end_list[i]],weight_matrix[start_list[i]:end_list[i]]):
                
            delta_local_cuts[i]+=weight*(2*spins[i]-1)*(2*spins[j]-1)
            curr_score+=weight*(spins[i]+spins[j]-2*spins[i]*spins[j])

    curr_score/=2   
    best_score=curr_score
    
    flag=True
    
    while flag:
        arg_gain=np.argsort(-delta_local_cuts)
        flag=False
        for v in arg_gain:
            
            if delta_local_cuts[v]<=0:
                flag=False
                break
                    
            curr_score+=delta_local_cuts[v]
            delta_local_cuts[v]=-delta_local_cuts[v]

            for u,weight in zip(adj_matrix[start_list[v]:end_list[v]],
                                 weight_matrix[start_list[v]:end_list[v]]):

                delta_local_cuts[u]+=weight*(2*spins[u]-1)*(2-4*spins[v])

            spins[v] =1-spins[v]
            flag=True
            break
                  
    return curr_score,spins

In [5]:
@njit
def tabu(graph,spins,tabu_tenure,max_steps):
    adj_matrix, weight_matrix, start_list, end_list=graph
    
    n=len(start_list)
    delta_local_cuts=np.zeros(n)
    
    tabu_list=np.ones(n)*-10000
    curr_score=0
    for i in range(n):
        for j,weight in zip(adj_matrix[start_list[i]:end_list[i]],
                  weight_matrix[start_list[i]:end_list[i]]):
                
            delta_local_cuts[i]+=weight*(2*spins[i]-1)*(2*spins[j]-1)
            curr_score+=weight*(spins[i]+spins[j]-2*spins[i]*spins[j])
            
#     print(delta_local_cuts)

    curr_score/=2    
    best_score=curr_score

    for t in range(max_steps):
        arg_gain=np.argsort(-delta_local_cuts)
        for v in arg_gain:
            if (t-tabu_list[v]> tabu_tenure) or (best_score < curr_score + delta_local_cuts[v]):

                tabu_list[v] = t

                curr_score+=delta_local_cuts[v]
                delta_local_cuts[v]=-delta_local_cuts[v]
                
                for u,weight in zip(adj_matrix[start_list[v]:end_list[v]],
                                     weight_matrix[start_list[v]:end_list[v]]):

                    delta_local_cuts[u]+=weight*(2*spins[u]-1)*(2-4*spins[v])

                spins[v] = 1-spins[v]

                break

                
        best_score=max(curr_score,best_score)
    return best_score,None
    
    

In [6]:
# graph_save_loc=f'_graphs/testing/ER_{"15-20"}spin_p{0.15}_50graphs.pkl'
# graphs=load_graph_set(graph_save_loc)

In [48]:
test_dataset=GraphDataset('data/validation/Uniform Random-3-SAT',ordered=True)

In [58]:
tabu_cuts=[]
for i in range(len(test_dataset)):
#     flatten_graph(graph)
    graph=test_dataset.get()
    g=flatten_graph(graph)
    spins= np.random.randint(2, size=graph.shape[0])
#     print(spins)
    cut,spins=tabu(g,spins,tabu_tenure=30,max_steps=graph.shape[0]*2)
    
    tabu_cuts.append(cut)
    
print('Mean Tabu Cut:',sum(tabu_cuts)/len(tabu_cuts))

Mean Tabu Cut: 5738.54


In [59]:
mca_cuts=[]
sg_cuts=[]
for i in range(len(test_dataset)):
#     flatten_graph(graph)
    graph=test_dataset.get()
    g=flatten_graph(graph)
    spins= np.random.randint(2, size=graph.shape[0])
#     print(spins)
    cut,spins=mca(g,spins)
    sg_cut,sg_spins=standard_greedy(g)
    
    mca_cuts.append(cut)
    sg_cuts.append(sg_cut)

In [60]:
sum(mca_cuts)/len(mca_cuts)

5736.5

In [61]:
sum(sg_cuts)/len(mca_cuts)

5739.6

In [ ]:
_spins=2*spins-1

In [ ]:
(0.25 * np.sum(graph * (1 - np.outer(_spins,_spins))))

In [ ]:
mca_cuts

In [ ]:
cuts=[]
for i in range(len(test_dataset)):
#     flatten_graph(graph)
    graph=test_dataset.get()
    
    cut,spins=standard_greedy(flatten_graph(graph))
    cuts.append(cut)
#     print()
    

In [ ]:
sum(cuts)/len(cuts)

In [ ]:
cuts